In [ ]:
'''The function below responds to an incoming text message from the user. It is designed to provide the user with trading signals
and other information on a requested stock. If the user says Hello/Bye the function will respond
with a hi/goodbye. In addition to that, any other text messages from the user will prompt an automated response of:

Hi, if you would like to know trading signals about a specific stock, please reply back with XTSE/ticker name, start_date 
and end_date. 
Please follow this format: XTSE/BMO, 2019-12-31, 2021-1-25 
Please do not forget the commas!
Please provide a min of 33 days difference between start & end dates.
Please note: Info will only be provided for stocks on XTSE.

If the user follows the specified format as stated above, the function will take the ticker and date information and 
calculate the RSI, Stochastic indicator, simple moving average and the Bollinger Bands. It will then reply to 
the customer with all the indicators, the high, low & closing prices for the end date and where the stock price is moving
with respective to the Bollinger bands on the requested end date  (ie. upper, lower etc.).

For this assignment, I have made use of the starter code provided by Twilio.
'''

In [ ]:
from flask import Flask, request, redirect
import requests
from twilio.twiml.messaging_response import MessagingResponse
%env quandl_key = raPgkJH_w8qLF5txbLAC
quandl.ApiConfig.api_key = %env quandl_key
import talib
import numpy as np


app = Flask(__name__)

@app.route("/sms", methods=['GET', 'POST'])
def incoming_sms():
    '''The function takes a message from the user either in the form of Hello, Bye or XTSE/ticker name, start_date, end_date
    and sends stock information & trading signals for the requested stock.'''
    body = request.values.get('Body', None)
    resp = MessagingResponse()
    
    if body == 'Hello':
        resp.message("Hi!")
    elif body == 'Bye':
        resp.message("Goodbye")
    # Sample code ends here
    try:
        x = body.split(',')
        y = quandl.get(x[0], start_date = x[1], end_date = x[2], column_index = 4)
    except:
        resp.message('''Hi, if you would like to know trading signals about a specific stock, please reply back with XTSE/ticker name, start_date and end_date. 
Please follow this format: XTSE/BMO, 2019-12-31, 2021-1-25 
Please do not forget the commas!
Please provide a min of 33 days difference between start & end dates.
Please note: Info will only be provided for stocks on XTSE.''')
        # I placed a 33 day requirement b/c STOCH doesn't provide data if its less than 33 days.
    else:
        close = y['Close'].values
        # Calculating RSI
        from talib import RSI
        output = RSI(close)
        global RSI_quote
        if output[-1] >= 70:
            RSI_quote = 'RSI: '+ str(round(output[-1], 4)) + ' The stock is overbought(bearish). It is indicating a sell signal.'
        elif output[-1] <= 30:
            RSI_quote = 'RSI: '+ str(round(output[-1], 4)) + ' The stock is oversold(bullish). It is indicating a buy signal.'
        else:
            RSI_quote = 'RSI: '+ str(round(output[-1], 4)) + ' The stock is neutral. It is not indicating a buy/sell signal.'
         
        # Calculating Stochastic
        from talib import STOCH
        h = quandl.get(x[0], start_date = x[1], end_date = x[2], column_index = 2)
        high = h['High'].values
        l = quandl.get(x[0], start_date = x[1], end_date = x[2], column_index = 3)
        low = l['Low'].values
        STOCH(high, low, close)
        s = STOCH(high, low, close)
        stoch_k = s[0][-1]
        if stoch_k >= 80:
            STOCH_quote = '%K: ' + str(round(stoch_k,4))+ ' The stock is overbought(bearish). It is indicating a sell signal.'
        elif stoch_k <= 20:
            STOCH_quote = '%K: ' + str(round(stoch_k,4)) + ' The stock is oversold(bullish). It is indicating a buy signal.'
        else:
            STOCH_quote = '%K: ' + str(round(stoch_k,4)) + ' The stock is neutral. It is not indicating a buy/sell signal.'
        
        # Calculating SMA & BBANDS
        from talib import BBANDS, SMA
        m = SMA(close)
        SMA_quote = 'The simple moving average price on end date was:' + str(np.round(m[-1], 4))
        bb = BBANDS(m)
        bb_u = bb[0][-1]
        bb_m = bb[1][-1]
        bb_l = bb[2][-1]
        c_hist = close[-5:]
        c = close[-1]
        if c < bb_u and c > bb_m:
            bb_quote = 'The stock price on end date was moving in the upper band. Here are the closing prices from previous 5 days.' + str(c_hist)
        elif c > bb_u:
            bb_quote = 'The stock price on end date was moving above the upper band. Here are the closing prices from previous 5 days.' + str(c_hist)
        elif c < bb_m and c > bb_l:
            bb_quote = 'The stock price on end date was moving in the lower band. Here are the closing prices from previous 5 days.' + str(c_hist)
        elif c < bb_l:
            bb_quote = 'The stock price on end date was moving below the lower band. Here are the closing prices from previous 5 days.' + str(c_hist)
    
        resp.message("Hi! The high, low and closing price on "+ x[2] + " for the stock " + x[0] + " was: "+ str([high[-1], low[-1], close[-1]]) + " Here are your trading signals for the requested stock: Relative Strength Index: "+ str(RSI_quote) + " Slow Stochastic Indicator:" + str(STOCH_quote) +' Simple Moving Average:' + str(SMA_quote) + ' Bollinger Bands:' + str(bb_quote))
 
    return str(resp)

if __name__ == "__main__":
    app.run()